# Data Validation and Transformation Workflow

As part of the assignment and as an additional add-on/research the goal of this task was to build a system that validates and transforms order data using a GenAI model. processed a JSON file of synthetic order transactions, validated each transaction with the model.

Leveraged the Meta LLaMA 3.3 70B Instruct model for this task, integrating it seamlessly via NVIDIA Enterprise’s robust API. This setup not only ensured high-quality outputs but also delivered exceptional execution speed, demonstrating the model's capability to handle complex tasks efficiently in real-time.

### Advantages if implemented

* Context-Aware work activity
* Automation: Automated workflows
* Multi-Task Capability

In [31]:
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps xformers trl peft accelerate bitsandbytes
!pip install -U langchain
!pip install -U langchain-community
!pip install -U sentence-transformers

!pip install -U faiss-gpu

  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-cpqtytev/unsloth_9210f8597dec4acd9f30f2a88b5a7d4a
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-cpqtytev/unsloth_9210f8597dec4acd9f30f2a88b5a7d4a
  Resolved https://github.com/unslothai/unsloth.git to commit b4c48d9c5e78203909495bf9beaa29a5c9aeaeeb
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 MB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 23.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 43.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 76.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 113.8/113.8 kB 7.2 MB/s eta 0:00:00
   ━━━━

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.3/15.3 MB 46.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.3/51.3 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 39.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 7.9 MB/s eta 0:00:00


In [26]:
import json
from openai import OpenAI

client = OpenAI(
    base_url="https://integrate.api.nvidia.com/v1",
    api_key="API-KEY"
)

def validate_and_transform_order(order_data):
    """
    Validate and transform order data using the GenAI model.
    Args:
        order_data (dict): The order data to validate and transform.
    Returns:
        dict: The parsed response content from the GenAI model.
    """
    prompt = f"""
    You are a data validation assistant. Validate the following order data:
    {json.dumps(order_data, indent=2)}

    Tasks:
    1. Ensure 'user_id', 'order_id', and 'order_value' are present and correctly typed.
    2. Verify 'order_value' equals the sum of (quantity * price_per_unit) for all items. If not, suggest corrections.
    3. Extract the fields 'user_id', 'order_value', and 'order_timestamp'.
    4. Highlight any errors or inconsistencies and propose fixes.
    Provide your response in JSON format.
    """
    completion = client.chat.completions.create(
        model="meta/llama-3.3-70b-instruct",
        messages=[{"role": "user", "content": prompt}],
        temperature=0.2,
        top_p=0.7,
        max_tokens=1024
    )

    response_content = completion.choices[0].message.content
    return response_content

input_file_path = "synthetic_order_data.json"
output_file_path = "validated_order_data.json"

with open(input_file_path, "r") as infile:
    orders = json.load(infile)

output_data = []
for order in orders:
    raw_response = validate_and_transform_order(order)
    output_data.append({
        "original_order": order,
        "genai_response": raw_response
    })

with open(output_file_path, "w") as outfile:
    json.dump(output_data, outfile, indent=2)

print(f"Validated data has been saved to {output_file_path}")


Validated data has been saved to validated_order_data.json


**NOTE** : The synthetic data contained 10-15 transactions , with a mix of noise and good data which was used to validate the responses from the model

In the Result json file the gen ai responses were 100% accurate ,it was successfully able to identify all validation checks as mentioned in assignement and in addition to that it was also able to capture some context rich information like payment methods,locations etc..

**Doubts**

* In Cloud environments integrating this technique to pipeline data validation check
* How can this technology/use case can be implemented real-time in real working environments
* Additional Data Validation checks to compute the accuracy
* How feasible is this for real-time use cases

### Additional Fine Tuning Explorations

In [35]:
from unsloth import FastLanguageModel
import torch
from datasets import load_dataset
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

from langchain.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [36]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "meta-llama/Meta-Llama-3.1-8B-Instruct",  #meta-llama/Meta-Llama-3.1-405B meta-llama/Meta-Llama-3-8B-Instruct
    max_seq_length = 2048,
    dtype = None,
    load_in_4bit = True,
    token = hf_token,
)


In [ ]:
# Apply LoRA (Low-Rank Adaptation) adapters to the model for parameter-efficient fine-tuning
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj"],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
)